In [1]:
from dask.distributed import Client, progress
c = Client('10.0.1.231:8786')
c

/opt/conda/lib/python3.8/site-packages/distributed/client.py:1146: VersionMismatchWarning: Mismatched versions found

+-------------+---------------+---------------+---------------+
| Package     | client        | scheduler     | workers       |
+-------------+---------------+---------------+---------------+
| blosc       | 1.10.2        | 1.9.2         | 1.10.2        |
| dask        | 2021.05.1     | 2021.05.0     | 2021.05.1     |
| distributed | 2021.05.1     | 2021.05.0     | 2021.05.1     |
| lz4         | 3.1.3         | 3.1.1         | 3.1.1         |
| msgpack     | 1.0.2         | 1.0.0         | 1.0.0         |
| python      | 3.8.6.final.0 | 3.8.0.final.0 | 3.8.0.final.0 |
+-------------+---------------+---------------+---------------+
Notes: 
-  msgpack: Variation is ok, as long as everything is above 0.6
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


Client Scheduler: tcp://10.0.1.231:8786 Dashboard: http://10.0.1.231:8787/status,Cluster Workers: 47 Cores: 47 Memory: 45.13 GiB


In [2]:
import json 

import pandas as pd 
import requests
import re 
import time

headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.110 Safari/537.36',
        'Connection':'close'}


import time

from functools import reduce

In [3]:
def get_hole(pid):
    url ='https://pkuhelper.pku.edu.cn/services/pkuhole/api.php?action=search&pagesize=50&page=1&keywords={}&PKUHelperAPI=3.0&jsapiver=201027113050-450368&user_token=4x4wsqhf1i116ofgyqcjubkkcwiof2vi'.format(pid)
    try:
        res = requests.get(url,headers=headers)
        res.raise_for_status()
        temp=json.loads(res.text)
        return temp['data']
    except (requests.HTTPError,json.JSONDecodeError) as err:
        return []
    

def concat_hole(data):
    return data

In [5]:
import signal
import time
def set_timeout(num, callback):
    def wrap(func):
        def handle(signum, frame): # 收到信号 SIGALRM 后的回调函数，第一个参数是信号的数字，第二个参数是the interrupted stack frame.
            raise RuntimeError
        def to_do(*args, **kwargs):
            try:
                signal.signal(signal.SIGALRM, handle) # 设置信号和回调函数
                signal.alarm(num) # 设置 num 秒的闹钟
                #print('start alarm signal.')
                r = func(*args, **kwargs)
                #print('close alarm signal.')
                signal.alarm(0) # 关闭闹钟
                return r
            except (RuntimeError,ConnectionError) as e:
                callback()
        return to_do
    return wrap
def after_timeout(): # 超时后的处理函数
    return []
@set_timeout(1, after_timeout) # 限时 1 秒超时
def get_result(future): # 要执行的函数
    try:
        return future.result()
    except ConnectionError:
        return []


In [6]:
pn=2199827 #最近的pid
lens=200

In [ ]:
state=True
while state:
    A=c.map(get_hole,range(pn-lens,pn))
    D=c.scatter(A)
    B=c.submit(concat_hole,D)
    st=time.time()
    C=B.result()
    ed=time.time()
    print('1.%s s'%(ed-st))
    ll=list(map(get_result,C))
    while None in ll:
        ll.remove(None)
    res=reduce(lambda x,y:x+y,ll)
    ed2=time.time()
    print('2.%s s'%(ed2-ed))
    pd.DataFrame(res).to_csv('./hole_%s.csv'%pn)
    pn-=lens
    if pn<2100000:
        state=False